# Knowledge Distillation in MHIST Dataset

In [213]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Utilities import *

In [266]:
import cv2
import os
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt

def load_images_from_folder(folder):
    images = []
    file_names = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
            file_names.append(filename)
    return images, file_names

path = "C:/Users/khoda/Downloads/images"
data, file_names = load_images_from_folder(path)
CSVfile = "C:/Users/khoda/Downloads/annotations.csv"

labels = pd.read_csv(CSVfile, usecols = [1])
Partitions = pd.read_csv(CSVfile, usecols = [3])
labels = labels.to_numpy()
Partitions = Partitions.to_numpy()
X_train = []
X_test = []
y_train = []
y_test = []
for i in range(len(data)):
    if Partitions[i] == 'train':
        X_train.append(data[i])
        if (labels[i] == 'SSA'):
            y_train.append(0)
        if (labels[i] == 'HP'):
            y_train.append(1)
    if Partitions[i] == 'test':
        X_test.append(data[i])
        if (labels[i] == 'SSA'):
            y_test.append(0)
        if (labels[i] == 'HP'):
            y_test.append(1)

#for i in range(len(X_train)):
#    X_train[i] = X_train[i].reshape(1,224,224,3)

#for i in range(len(X_test)):
#    X_test[i] = X_test[i].reshape(1,224,224,3)

print(y_test)
#print(X_train[1].shape[2])


[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 

In [321]:
def train_and_evaluate_2(model,trainingData, testingData, trainingLabel, testLabel):
    """Perform training and evaluation for the teacher model model.

    Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
        images, and labels.
    """

    # your code start from here for step 4
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(1, NUM_EPOCHS + 1):
        # Run training.
        print('Epoch {}: '.format(epoch), end='')
        for i in range(len(trainingLabel)):
            with tf.GradientTape() as tape:
                loss_value = compute_loss(model,trainingData[i],trainingLabel[i])
            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Run evaluation.
        num_correct = 0
        num_total = builder.info.splits['test'].num_examples
        for i in range(len(testLabel)):
            # your code start from here for step 4
            num_correct += compute_num_correct(model,testingData[i],testLabel[i])[0]
        print("Class_accuracy: " + '{:.2f}%'.format(
            num_correct / num_total * 100))

def compute_loss(model,images, labels):
    """Compute subclass knowledge distillation teacher loss for given images
        and labels.

    Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

    Returns:
    Scalar loss Tensor. 
    """
    subclass_logits = model(images, training=True)
    print(subclass_logits.shape[0], subclass_logits.shape[1],subclass_logits.shape[2],subclass_logits.shape[3])
    cross_entropy_loss_value = tf.math.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=subclass_logits,labels=labels))
    return cross_entropy_loss_value

def train_and_evaluate_using_KD(studentModel, teacherModel,trainingData, testingData, trainingLabel, testLabel, alpha, temprature):
    """Perform training and evaluation for the teacher model model.

    Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
        images, and labels.
    """

    # your code start from here for step 4
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(1, NUM_EPOCHS + 1):
        # Run training.
        print('Epoch {}: '.format(epoch), end='')
        for i in range(len(trainingLabel)):
            with tf.GradientTape() as tape:
                loss_value = compute_student_loss_using_KD(studentModel, teacherModel,trainingData[i],trainingLabel[i], alpha, temprature)
            grads = tape.gradient(loss_value, studentModel.trainable_variables)
            optimizer.apply_gradients(zip(grads, studentModel.trainable_variables))

        # Run evaluation.
        num_correct = 0
        num_total = builder.info.splits['test'].num_examples
        for i in range(len(testLabel)):
            # your code start from here for step 4
            num_correct += compute_num_correct(studentModel,testingData[i],testLabel[i])[0]
        print("Class_accuracy: " + '{:.2f}%'.format(
            num_correct / num_total * 100))

In [336]:
import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Input

ResNet= tf.keras.applications.resnet_v2.ResNet50V2(
    include_top = False,
    input_shape=None,
    weights='imagenet',
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)
studenModel2 = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape= None,
    alpha=1.0,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=2,
    classifier_activation='softmax'
)
model = models.Sequential()
model.add(ResNet)
#model.add(Flatten())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=1, input_shape=(1, 7,7,2048)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=1))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128 , activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Activation('relu'))

train_and_evaluate_2(model,X_train,X_test, y_train, y_test)
#teacherTestAcc = testModel(teacherModel2,X_test, y_test)
#train_and_evaluate_using_KD(studenModel2,teacherModel2,X_train,X_test, y_train, y_test, 0.5,4)
#studentTestAcc = testModel(studenModel2,X_test)



ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)